# 🚀 Train Your First Custom Wake Word with Nanowakeword!

Welcome to the official tutorial for **Nanowakeword**! 

In this notebook, we will guide you through the entire process of training a high-performance, custom wake word model from scratch. You don't need any pre-existing data—we will download everything we need and let Nanowakeword's intelligent engine do the heavy lifting.

**Our goal:** Go from zero to a ready-to-use wake word model in just a few simple steps. Let's get started!

**Installation**

In [ ]:
# @title Step 1: Install Nanowakeword
# We install the full [train] package to get all the necessary dependencies.

# ! pip install --no-cache-dir "nanowakeword[train]==1.3.4" # will comeing
! pip install "nanowakeword[train] @ git+https://github.com/arcosoph/nanowakeword.git"
! pip install piper-tts

print("Installation complete!")

## Step 2: Prepare the Dataset

A great model starts with great data. For this tutorial, we will:
1.  **Download** open-source noise and Room Impulse Response (RIR) datasets.
2.  **Organize** all your project files within a clean, well-structured folder hierarchy for better clarity and maintainability.

In [2]:
import shutil
import os, requests
from tqdm import tqdm
from datasets import load_dataset, Audio
from huggingface_hub import hf_hub_download
from concurrent.futures import ThreadPoolExecutor

# RIR data
# Disable HF symlink warning
os.environ["HF_HUB_DISABLE_SYMLINKS_WARNING"] = "1"
# Output directory
output_dir = "data/rir"
os.makedirs(output_dir, exist_ok=True)

# Load dataset (streaming, no decode)
dataset = load_dataset(
    "davidscripka/MIT_environmental_impulse_responses",
    split="train",
    streaming=True
)
dataset = dataset.cast_column("audio", Audio(decode=False))

repo_id = "davidscripka/MIT_environmental_impulse_responses"

for row in tqdm(dataset):
    hf_path = row["audio"]["path"]
    filename = os.path.basename(hf_path)
    save_path = os.path.join(output_dir, filename)

    if os.path.exists(save_path):
        continue

    # Download from HF cache (or repo)
    local_file = hf_hub_download(
        repo_id=repo_id,
        filename=hf_path.split("/")[-2] + "/" + filename,  # keep subfolder
        repo_type="dataset"
    )

    # Cross-drive safe copy
    shutil.copy(local_file, save_path)


# noice data
out_dir = "data/background_noise"
os.makedirs(out_dir, exist_ok=True)

# 1) Get file list from GitHub API
api = "https://api.github.com/repos/karolpiczak/ESC-50/contents/audio"
files = [f["name"] for f in requests.get(api).json() if f["name"].endswith(".wav")]

base = "https://raw.githubusercontent.com/karolpiczak/ESC-50/master/audio/"

def dl(fname):
    path = os.path.join(out_dir, fname)
    if os.path.exists(path): return
    r = requests.get(base + fname, stream=True)
    if r.status_code == 200:
        with open(path, "wb") as f:
            for c in r.iter_content(8192):
                f.write(c)

# 2) Parallel download (SUPERFAST)
with ThreadPoolExecutor(max_workers=16) as ex:
    list(tqdm(ex.map(dl, files), total=len(files)))


100%|██████████| 1000/1000 [04:06<00:00,  4.06it/s]


## Step 3: Configure and Train the Model

Now for the fun part! We will create a `config.yaml` file and then run the Nanowakeword training command.

In [ ]:
# @title Step 3.1: Create the "NanoWakeWord" Configuration File
import yaml

# =========================================================
# Project & Data Paths
# Define the project name and the locations of all data assets.
# =========================================================
config_dict = {
    "model_name": "arcosoph_A_v1",
    "output_dir": "./trained_models",

    # -----------------------------------------------------
    # Source audio data paths (raw .wav files)
    # -----------------------------------------------------
    "positive_data_path": "./data/generated_positive",
    "negative_data_path": "./data/generated_negative",
    "background_paths": ["./data/background_noise"],
    "rir_paths": ["./data/rir"],

    # =====================================================
    # Model Architecture
    # Configure the structure and properties of the neural network.
    # =====================================================
    "model_type": "dnn", # or other architectures like `RNN`..etc
    "layer_size": 128,
    "n_blocks": 3,
    "dropout_prob": 0.2,   # To reduce overfitting

    # =====================================================
    # Training & Optimizer Settings
    # Hyperparameters that control the training loop
    # =====================================================
    "steps": 20000,
    "batch_size": 128,
    "optimizer_type": "adamw",
    "learning_rate_max": 0.001,
    "lr_scheduler_type": "onecycle",
    "weight_decay": 0.01,
    "momentum": 0.9,
    "num_workers": 3,

    # =====================================================
    # Feature Manifest
    # Paths to the generated .npy feature files.
    # You can add unlimited additional DATA(.npy) sources with unique keys
    # =====================================================
    "feature_manifest": {
        "targets": {
            # positive / wakeword features
            "t": "./trained_models/arcosoph_A_v1/features/positive_features_train.npy"
        },
        "negatives": {
            # hard negative features
            "n": "./trained_models/arcosoph_A_v1/features/hard_negative_features.npy"
            # "my_other_negatives": "/path/to/other_negatives.npy"
        },
        "backgrounds": {
            # pure background / ambient noise features
            "b": "./trained_models/arcosoph_A_v1/features/pure_noise_features.npy"
            # For example, adding a large external noise dataset:
            # "openwakeword_noise": "./openwakeword_features_ACAV100M_2000_hrs_16bit.npy"
        }
    },

    # =====================================================
    # Synthetic Data Generation
    # Parameters for generating audio clips using TTS
    # =====================================================
    "generate_positive_samples": 2500, # How many positive samples generate
    "generate_negative_samples": 5000, # How many positive samples generate

    "include_input_words": 0.7,
    "include_partial_phrase": 0.5,

    "augmentation_batch_size": 16,
    "feature_gen_cpu_ratio": 1.0,

    # =====================================================
    # Feature Generation Manifest
    # Controls feature extraction & augmentation
    # =====================================================
    "feature_generation_manifest": {

        # ------------------ Positive Wakeword ------------------
        "positive_wakeword": {
            "input_audio_dirs": ["./data/generated_positive"],
            "output_filename": "positive_features_train.npy",
            "use_background_noise": True,
            "use_rir": True,
            "augmentation_rounds": 11
            # augmentation_settings: default
        },

        # ------------------ Hard Negatives ------------------
        "hard_negatives": {
            "input_audio_dirs": ["./data/generated_negative"],
            "output_filename": "hard_negative_features.npy",
            "use_background_noise": True,
            "use_rir": True,
            "augmentation_rounds": 11
        },

        # ------------------ Pure Ambient Noise ------------------
        "pure_ambient_noise": {
            "input_audio_dirs": ["./data/background_noise"],
            "output_filename": "pure_noise_features.npy",
            "use_background_noise": False,
            "augmentation_rounds": 3,
            "augmentation_settings": {
                "PitchShift": 0.6,
                "AddBackgroundNoise": 0.0,
                "Gain": 1.0,
                "RIR": 0.5
            }
        }
    },

    # =====================================================
    # Advanced & Miscellaneous Settings
    # =====================================================
    "checkpoint_averaging_top_k": 5,
    "checkpointing": {
        "enabled": True,
        "interval_steps": 1000,
        "limit": 2
    },

    "early_stopping_patience": 0, # If you want 
    "main_delta": 0.0001,
    "WARMUP_STEPS": 1500,

    "onnx_opset_version": 17,
    "show_training_summary": True,
    "debug_mode": True,
    "ema_alpha": 0.01,

    # =====================================================
    # Pipeline Control Switches
    # Master switches to enable or disable stages
    # =====================================================
    "generate_clips": True,
    "transform_clips": True,   # ⚠️ Don't forget to turn it off after finished.
    "train_model": True,
    "overwrite": True
}

# =========================================================
# Write config to YAML
# =========================================================
config_path = "./config.yaml"
with open(config_path, "w") as f:
    yaml.dump(config_dict, f, default_flow_style=False, sort_keys=False)

print(
    "⚠️[NOTE] Set `transform_clips = False` and re-run this cell unless feature (.npy) re-generation is required.\n"
)
print("config.yaml written successfully")

⚠️[NOTE] Set `transform_clips = False` and re-run this cell unless feature (.npy) re-generation is required.

config.yaml written successfully


**Run Training!**

In [ ]:
# @title Step 3.2: Run the Magic Command! 🚀
# This command will do everything: augment data, extract features, and train the model.
# It might take some time depending on the hardware (especially on a CPU).

from nanowakeword.trainer import train 

args_list = [
    '--config_path', f'{config_path}',
]

print("Starting NanoWakeWord training...")

try:
    train(args_list)
    print("\n\nCONGRATULATIONS! (✿◕‿◕✿)")
    print("Your custom wake word model has been successfully trained!")

except Exception as e:
    print(f"\nAn error occurred during training: {e}")

## What's Next?

You have successfully trained your own custom wake word model!

You can now download the `.onnx` file from the `trained_models` directory (check the file browser on the left) and use it in your own applications.

For more advanced topics, such as using your own datasets or fine-tuning the configuration, please check out our full documentation on **[GitHub](https://github.com/arcosoph/nanowakeword)**.

---
## Step 4: Save Your Model to Google Drive

The final step is to save your trained model and performance graph to a safe and accessible place. Instead of a slow direct download, we will save the files directly to your Google Drive. This process is almost instantaneous.

Run the cells below to:
1.  Connect your Google Drive account.
2.  Copy all the trained files into a new folder named `nanowakeword_models` in your Drive.

In [ ]:
# @title Step 4.1: Connect to Google Drive
# This will ask for your permission to access your Google Drive.

from google.colab import drive

try:
    drive.mount('/content/drive')
    print("\nGoogle Drive connected successfully!")
except Exception as e:
    print(f"An error occurred while connecting to Google Drive: {e}")

In [ ]:
# @title Step 4.2: Copy Final Model and Artifacts to Google Drive 📂

import os
import shutil

# --- Configuration ---
# Get model_name and output_dir from the config_dict defined earlier
model_name = config_dict.get("model_name", "my_model")
output_dir = config_dict.get("output_dir", "./trained_models")

# --- Source and Destination Paths ---
# The source project directory containing all generated files
source_project_dir = os.path.join(output_dir, model_name)

# The destination folder in your Google Drive
drive_destination_dir = f"drive/MyDrive/nanowakeword_models/{model_name}"

# --- Start Copy Process ---
print("Starting the process to copy trained files to Google Drive...")

# Check if the source directory exists
if not os.path.exists(source_project_dir):
    print(f"\nERROR: Source directory not found at '{source_project_dir}'")
    print("This indicates that the training process did not create the expected output folder.")
    print("Please ensure the training step completed successfully before running this cell.")
else:
    # If an old folder exists in Drive, remove it to ensure a clean copy
    if os.path.exists(drive_destination_dir):
        print(f"🔄 Found an existing folder in Drive. Removing it for a fresh copy: '{drive_destination_dir}'")
        shutil.rmtree(drive_destination_dir)

    # --- Copy the entire project folder ---
    # This is much simpler and more reliable than copying individual files.
    # It preserves the professional directory structure.
    try:
        shutil.copytree(source_project_dir, drive_destination_dir)
        
        print("\n" + "="*50)
        print("✅ SUCCESS! All files have been saved to your Google Drive.")
        print("="*50)
        print(f"\nYour complete project, including the model and performance graphs, can be found in:")
        print(f"➡️ '{drive_destination_dir}'")
        
        # Optional: List the contents of the new folder in Drive for verification
        print("\nContents of the saved folder:")
        for root, dirs, files in os.walk(drive_destination_dir):
            level = root.replace(drive_destination_dir, '').count(os.sep)
            indent = ' ' * 4 * (level)
            print(f"{indent}{os.path.basename(root)}/")
            sub_indent = ' ' * 4 * (level + 1)
            for f in files:
                print(f"{sub_indent}{f}")

    except Exception as e:
        print(f"\nERROR: An unexpected error occurred during the copy process.")
        print(f"Details: {e}")